In [56]:
# Importação das bibliotecas necessárias

import pandas as pd
import numpy as np
import glob
import os
import re
import csv
from sqlalchemy import create_engine
from datetime import datetime
import locale
from time import time

In [57]:
# Estabelecimento de conexão com o banco de dados
# para extração das informações das tabelas necessárias
start = time()

database_config = {
    "host": "cat42.cohe0l2miaal.us-east-1.rds.amazonaws.com",
    "user": "master",
    "password": "dms191411",
    "database": "cat"
}

# Create a connection to the MySQL database
engine = create_engine(f"mysql+pymysql://{database_config['user']}:{database_config['password']}@{database_config['host']}/{database_config['database']}")

In [58]:
# Definição da loja a ser utilizada

loja = 63

In [59]:
# Importação das planilhas necessárias para
# gerar os documentos da 000, 0150, 0200, 1050 e 1100

planilha = pd.read_excel(f'Loja_{loja}/ficha_3_loja_{loja}_v4.xlsb', sheet_name='ficha3', engine='pyxlsb')
planilha['DATA'] = pd.to_datetime(planilha['DATA'], format='%Y-%m-%d')
template = pd.read_excel('CFOP_e_template.xlsx', sheet_name='Template 150', dtype=str)
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,VLR_CONFR_PCAT,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,ALIQUOTA,VALOR,Valor Base Cálculo ICMS ST Retido Operação Anterior,Valor Complementar,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT
0,35191271605265000242590003519330916642468850,2019-12-23,5405,1,18,1,1,1,5.0,14.234097,...,NaN,4.0,11.40,18,19.96,0.0,NaN,0.0,0.0,0.0
1,35200171605265000242590006721510483174585808,2020-01-27,5405,1,18,1,1,1,NaN,NaN,...,NaN,NaN,NaN,18,20.68,0.0,NaN,0.0,0.0,0.0
2,35200171605265000242590006721510484210083682,2020-01-27,5405,15,18,1,1,1,NaN,NaN,...,NaN,NaN,NaN,18,16.96,0.0,NaN,0.0,0.0,0.0
3,35200171605265000242590006721510484210083682,2020-01-27,5405,16,18,1,1,1,NaN,NaN,...,NaN,NaN,NaN,18,16.96,0.0,NaN,0.0,0.0,0.0
4,35200171605265000242590006721510484210083682,2020-01-27,5405,17,18,1,1,1,NaN,NaN,...,NaN,NaN,NaN,18,16.96,0.0,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416286,35201271605265000242590003519331309428707136,2020-12-15,5405,1,935908,1,1,1,NaN,NaN,...,4.119925,NaN,NaN,12,21.91,0.0,NaN,0.0,0.0,0.0
416287,35201271605265000242590006721510835754036640,2020-12-15,5405,1,935908,1,1,1,NaN,NaN,...,4.120000,NaN,NaN,12,21.91,0.0,NaN,0.0,0.0,0.0
416288,35201271605265000242590006721510835754036640,2020-12-15,5405,3,935908,1,1,1,NaN,NaN,...,4.120000,NaN,NaN,12,21.91,0.0,NaN,0.0,0.0,0.0
416289,35201271605265000242590006721510844913169435,2020-12-20,5405,1,935908,1,1,1,NaN,NaN,...,4.120000,NaN,NaN,12,21.91,0.0,NaN,0.0,0.0,0.0


In [60]:
tabela_2 = pd.read_csv(f'Loja_{loja}/loja_{loja}_tabela_2_v4.csv')
tabela_2 = tabela_2.sort_values(by=['COD_ITEM', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                           ascending=[True, True, True, False])
tabela_2

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\2324544435.py:1: DtypeWarning: Columns (20,26) have mixed types. Specify dtype option on import or set low_memory=False.
  tabela_2 = pd.read_csv(f'Loja_{loja}/loja_{loja}_tabela_2_v4.csv')


,Unnamed: 0,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_EFD,...,Valor ICMS Operação,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Valor Base Cálculo ICMS Substituição Tributária,FONTE
214155,237644,35191271605265000242590003519330916642468850,2019-12-23,5405,1,18,1,1,1,NaN,...,0.0,2294 - DEPAKENE 250MG 25'S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
138866,153565,35200171605265000242590006721510483174585808,2020-01-27,5405,1,18,1,1,1,NaN,...,0.0,1006 - DEPAKENE 250MG 25S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
259076,287488,35200171605265000242590006721510484210083682,2020-01-27,5405,15,18,1,1,1,NaN,...,0.0,2669 - DEPAKENE 250MG 25S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
259077,287489,35200171605265000242590006721510484210083682,2020-01-27,5405,16,18,1,1,1,NaN,...,0.0,2669 - DEPAKENE 250MG 25S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
259078,287490,35200171605265000242590006721510484210083682,2020-01-27,5405,17,18,1,1,1,NaN,...,0.0,2669 - DEPAKENE 250MG 25S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131980,145944,35201271605265000242590003519331309428707136,2020-12-15,5405,1,935908,1,1,1,NaN,...,0.0,1006 - ATORVASTATINA 20MG 30S EMS,7896004729435.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
131982,145946,35201271605265000242590006721510835754036640,2020-12-15,5405,1,935908,1,1,1,NaN,...,0.0,1006 - ATORVASTATINA 20MG 30S EMS,7896004729435.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
131983,145947,35201271605265000242590006721510835754036640,2020-12-15,5405,3,935908,1,1,1,NaN,...,0.0,1006 - ATORVASTATINA 20MG 30S EMS,7896004729435.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
381472,424377,35201271605265000242590006721510844913169435,2020-12-20,5405,1,935908,1,1,1,NaN,...,0.0,46890 - ATORVASTATINA 20MG 30S EMS,7896004729435.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe


In [61]:
planilha['DESCRICAO'] = tabela_2['DESCRICAO']
planilha['CODIGO_BARRA'] = tabela_2['CODIGO_BARRA']
planilha['UNIDADE'] = tabela_2['UNIDADE']
planilha['N C M'] = tabela_2['N C M']
planilha['CEST'] = tabela_2['CEST']
planilha['CNPJ EMITENTE'] = tabela_2['CNPJ EMITENTE']
planilha['CNPJ DESTINATARIO'] = tabela_2['CNPJ DESTINATARIO']

In [62]:
planilha.head()

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,35191271605265000242590003519330916642468850,2019-12-23,5405,1,18,1,1,1,5.0,14.234097,...,0.0,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090.0,1300101.0,7.160527e+13,0.0
1,35200171605265000242590006721510483174585808,2020-01-27,5405,1,18,1,1,1,NaN,NaN,...,0.0,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090.0,1300101.0,7.160527e+13,0.0
2,35200171605265000242590006721510484210083682,2020-01-27,5405,15,18,1,1,1,NaN,NaN,...,0.0,0.0,0.0,Sal De Fruta Tradicional 100g,SEM GTIN,UN,30049029.0,1300301.0,7.160527e+13,0.0
3,35200171605265000242590006721510484210083682,2020-01-27,5405,16,18,1,1,1,NaN,NaN,...,0.0,0.0,0.0,Esogastro Ibp 20mg 84S,SEM GTIN,UN,30049099.0,1300301.0,7.160527e+13,0.0
4,35200171605265000242590006721510484210083682,2020-01-27,5405,17,18,1,1,1,NaN,NaN,...,0.0,0.0,0.0,Dexilant 60mg 60S Takeda,SEM GTIN,UN,30039033.0,0.0,7.160527e+13,0.0


In [63]:
planilha['DESCRICAO'] = planilha['DESCRICAO'].str.replace('"', "'")

In [64]:
planilha['N C M'] = pd.to_numeric(planilha['N C M'])
planilha['N C M'] = planilha['N C M'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CEST'] = pd.to_numeric(planilha['CEST'])
planilha['CEST'] = planilha['CEST'].astype('Int64').astype(str).replace('<NA>', np.nan)
planilha['CEST'] = planilha['CEST'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CNPJ EMITENTE'] = pd.to_numeric(planilha['CNPJ EMITENTE'])
planilha['CNPJ EMITENTE'] = planilha['CNPJ EMITENTE'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['QTD_CAT'] = planilha['QTD_CAT'].astype(str) + ',000'

planilha['COD_LEGAL_PCAT'] = pd.to_numeric(planilha['COD_LEGAL_PCAT'])
planilha['COD_LEGAL_PCAT'] = planilha['COD_LEGAL_PCAT'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['CFOP'] = pd.to_numeric(planilha['CFOP'])
planilha['CFOP'] = planilha['CFOP'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['UNIDADE'] = planilha['UNIDADE'].str.strip()

def format_number(value):
    return '{:,.2f}'.format(value).replace(',', '.')

planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,35191271605265000242590003519330916642468850,2019-12-23,5405,1,18,1,1,"1,000",5.0,14.234097,...,0.0,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00
1,35200171605265000242590006721510483174585808,2020-01-27,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00
2,35200171605265000242590006721510484210083682,2020-01-27,5405,15,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Sal De Fruta Tradicional 100g,SEM GTIN,UN,30049029,1300301,71605265000242,0.000000e+00
3,35200171605265000242590006721510484210083682,2020-01-27,5405,16,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Esogastro Ibp 20mg 84S,SEM GTIN,UN,30049099,1300301,71605265000242,0.000000e+00
4,35200171605265000242590006721510484210083682,2020-01-27,5405,17,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Dexilant 60mg 60S Takeda,SEM GTIN,UN,30039033,0,71605265000242,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416286,35201271605265000242590003519331309428707136,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Levoid 88mcg 30S Ache,7896658008801,UN,30043981,1300300,71605265000242,7.160527e+13
416287,35201271605265000242590006721510835754036640,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Fenobarbital U.Q 100mg 30S C1,7896006237372,UN,30049069,1300200,71605265000242,7.160527e+13
416288,35201271605265000242590006721510835754036640,2020-12-15,5405,3,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Alprazolam Leg 2mg 30S B1,7894916149143,UN,30049064,1300200,71605265000242,7.160527e+13
416289,35201271605265000242590006721510844913169435,2020-12-20,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Ceftriaxona Eur 1g Im 1ml,7891317007836,UN,30042059,1300200,71605265000242,7.160527e+13


In [65]:
query = f'SELECT * FROM tabela_efds_modelo55 WHERE LOJA = {loja}'

efd = pd.read_sql(query, engine)
efd

,ID,LOJA,DATA_EFD,DATA_ENTRADA,DATA_SAIDA,CHAVE_ORIGINAL,COD_DO_ITEM,CODPROF,CFOP,DESC_PROD,QUANTI_EFDS,EAN,NCM,ICMS,CEST
0,6397620,63,01-2020,02/01/2020,04/01/2020,35200171605265000323550020000033801163719826,1,168799,1409,Saxenda 6mg/Ml C/3,1,,30043929,18,13.003.01
1,6397621,63,01-2020,07/01/2020,07/01/2020,35200171605265000323550020000033821163973913,1,139360,1409,NESINA 25MG 30'S,1,7896641809170,30049099,18,13.001.01
2,6397622,63,01-2020,08/01/2020,08/01/2020,35200171605265000323550020000033841164048886,1,85175,1409,ASPIRINA PREV 100MG 30'S BAYER,2,7891106001946,30049024,18,13.001.01
3,6397623,63,01-2020,08/01/2020,08/01/2020,35200171605265000323550020000033841164048886,2,126011,1409,PRADAXA 150MG 30'S BOEHRINGER,1,7896026304702,30049059,18,13.001.00
4,6397624,63,01-2020,08/01/2020,09/01/2020,35200171605265000323550020000033861164063529,1,60426,1409,PROLOPA 200/50 30'S ROCHE,1,7896226500515,30049049,18,13.001.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143293,6541272,63,12-2020,,,35201271605265000242550030000000041420285330,,,,,,,,,
143294,6541273,63,12-2020,,,35201271605265000242550020000060621184489660,,,,,,,,,
143295,6541274,63,12-2020,,,35201271605265000242550020000060731184816878,,,,,,,,,
143296,6541275,63,12-2020,,,35201271605265000242550030000000011418319255,,,,,,,,,


In [66]:
planilha = planilha.drop_duplicates()
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,35191271605265000242590003519330916642468850,2019-12-23,5405,1,18,1,1,"1,000",5.0,14.234097,...,0.0,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00
1,35200171605265000242590006721510483174585808,2020-01-27,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00
2,35200171605265000242590006721510484210083682,2020-01-27,5405,15,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Sal De Fruta Tradicional 100g,SEM GTIN,UN,30049029,1300301,71605265000242,0.000000e+00
3,35200171605265000242590006721510484210083682,2020-01-27,5405,16,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Esogastro Ibp 20mg 84S,SEM GTIN,UN,30049099,1300301,71605265000242,0.000000e+00
4,35200171605265000242590006721510484210083682,2020-01-27,5405,17,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Dexilant 60mg 60S Takeda,SEM GTIN,UN,30039033,0,71605265000242,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416286,35201271605265000242590003519331309428707136,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Levoid 88mcg 30S Ache,7896658008801,UN,30043981,1300300,71605265000242,7.160527e+13
416287,35201271605265000242590006721510835754036640,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Fenobarbital U.Q 100mg 30S C1,7896006237372,UN,30049069,1300200,71605265000242,7.160527e+13
416288,35201271605265000242590006721510835754036640,2020-12-15,5405,3,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Alprazolam Leg 2mg 30S B1,7894916149143,UN,30049064,1300200,71605265000242,7.160527e+13
416289,35201271605265000242590006721510844913169435,2020-12-20,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Ceftriaxona Eur 1g Im 1ml,7891317007836,UN,30042059,1300200,71605265000242,7.160527e+13


In [67]:
planilha['CODIGO_BARRA'] = np.where(pd.notnull(planilha['CODIGO_BARRA']), planilha['CODIGO_BARRA'].astype(str).str.replace(r'\.0$', '', regex=True),
                                   np.nan)
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,35191271605265000242590003519330916642468850,2019-12-23,5405,1,18,1,1,"1,000",5.0,14.234097,...,0.0,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00
1,35200171605265000242590006721510483174585808,2020-01-27,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00
2,35200171605265000242590006721510484210083682,2020-01-27,5405,15,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Sal De Fruta Tradicional 100g,SEM GTIN,UN,30049029,1300301,71605265000242,0.000000e+00
3,35200171605265000242590006721510484210083682,2020-01-27,5405,16,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Esogastro Ibp 20mg 84S,SEM GTIN,UN,30049099,1300301,71605265000242,0.000000e+00
4,35200171605265000242590006721510484210083682,2020-01-27,5405,17,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Dexilant 60mg 60S Takeda,SEM GTIN,UN,30039033,0,71605265000242,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416286,35201271605265000242590003519331309428707136,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Levoid 88mcg 30S Ache,7896658008801,UN,30043981,1300300,71605265000242,7.160527e+13
416287,35201271605265000242590006721510835754036640,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Fenobarbital U.Q 100mg 30S C1,7896006237372,UN,30049069,1300200,71605265000242,7.160527e+13
416288,35201271605265000242590006721510835754036640,2020-12-15,5405,3,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Alprazolam Leg 2mg 30S B1,7894916149143,UN,30049064,1300200,71605265000242,7.160527e+13
416289,35201271605265000242590006721510844913169435,2020-12-20,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0,Ceftriaxona Eur 1g Im 1ml,7891317007836,UN,30042059,1300200,71605265000242,7.160527e+13


In [68]:
mes_ano = []
for date in planilha['DATA']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])

planilha['Data'] = mes_ano
planilha['Data'].unique()

array(['122019', '012020', '042020', '052020', '122020', '102019',
       '022020', '062020', '092020', '102020', '112020', '112019',
       '032020', '072020', '082020'], dtype=object)

In [69]:
planilha.replace('nan', np.nan, inplace=True)

In [70]:
planilha['DATA'].astype(str).iloc[0][8:]

'23'

In [71]:
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,35191271605265000242590003519330916642468850,2019-12-23,5405,1,18,1,1,"1,000",5.0,14.234097,...,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00,122019
1,35200171605265000242590006721510483174585808,2020-01-27,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00,012020
2,35200171605265000242590006721510484210083682,2020-01-27,5405,15,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Sal De Fruta Tradicional 100g,SEM GTIN,UN,30049029,1300301,71605265000242,0.000000e+00,012020
3,35200171605265000242590006721510484210083682,2020-01-27,5405,16,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Esogastro Ibp 20mg 84S,SEM GTIN,UN,30049099,1300301,71605265000242,0.000000e+00,012020
4,35200171605265000242590006721510484210083682,2020-01-27,5405,17,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Dexilant 60mg 60S Takeda,SEM GTIN,UN,30039033,0,71605265000242,0.000000e+00,012020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416286,35201271605265000242590003519331309428707136,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Levoid 88mcg 30S Ache,7896658008801,UN,30043981,1300300,71605265000242,7.160527e+13,122020
416287,35201271605265000242590006721510835754036640,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Fenobarbital U.Q 100mg 30S C1,7896006237372,UN,30049069,1300200,71605265000242,7.160527e+13,122020
416288,35201271605265000242590006721510835754036640,2020-12-15,5405,3,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Alprazolam Leg 2mg 30S B1,7894916149143,UN,30049064,1300200,71605265000242,7.160527e+13,122020
416289,35201271605265000242590006721510844913169435,2020-12-20,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Ceftriaxona Eur 1g Im 1ml,7891317007836,UN,30042059,1300200,71605265000242,7.160527e+13,122020


In [72]:
planilha['CEST'] = planilha['CEST'].replace('0', pd.NA)

# Second, add leading '0' to values with length equal to 6
planilha.loc[planilha['CEST'].str.len() == 6, 'CEST'] = '0' + planilha['CEST']

planilha['CEST']

0         1300101
1         1300101
2         1300301
3         1300301
4            <NA>
           ...   
416286    1300300
416287    1300200
416288    1300200
416289    1300200
416290       <NA>
Name: CEST, Length: 416291, dtype: object

In [73]:
planilha['N C M'] = planilha['N C M'].apply(lambda x: str(x).zfill(8) if len(str(x)) < 8 else str(x))
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,35191271605265000242590003519330916642468850,2019-12-23,5405,1,18,1,1,"1,000",5.0,14.234097,...,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00,122019
1,35200171605265000242590006721510483174585808,2020-01-27,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00,012020
2,35200171605265000242590006721510484210083682,2020-01-27,5405,15,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Sal De Fruta Tradicional 100g,SEM GTIN,UN,30049029,1300301,71605265000242,0.000000e+00,012020
3,35200171605265000242590006721510484210083682,2020-01-27,5405,16,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Esogastro Ibp 20mg 84S,SEM GTIN,UN,30049099,1300301,71605265000242,0.000000e+00,012020
4,35200171605265000242590006721510484210083682,2020-01-27,5405,17,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Dexilant 60mg 60S Takeda,SEM GTIN,UN,30039033,<NA>,71605265000242,0.000000e+00,012020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416286,35201271605265000242590003519331309428707136,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Levoid 88mcg 30S Ache,7896658008801,UN,30043981,1300300,71605265000242,7.160527e+13,122020
416287,35201271605265000242590006721510835754036640,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Fenobarbital U.Q 100mg 30S C1,7896006237372,UN,30049069,1300200,71605265000242,7.160527e+13,122020
416288,35201271605265000242590006721510835754036640,2020-12-15,5405,3,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Alprazolam Leg 2mg 30S B1,7894916149143,UN,30049064,1300200,71605265000242,7.160527e+13,122020
416289,35201271605265000242590006721510844913169435,2020-12-20,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Ceftriaxona Eur 1g Im 1ml,7891317007836,UN,30042059,1300200,71605265000242,7.160527e+13,122020


In [74]:
planilha.columns

Index(['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'COD_ITEM', 'IND_OPER',
       'SUB_TIPO', 'QTD_CAT', 'QTD_INI', 'ICMS_INI', 'QTD_ent1_devolv_ent',
       'ICMS_TOT_0', 'ICMS_TOT_ent_unit', 'ULT_ICMS_TOT_ent_unit',
       'ICMS_TOT_1', 'qtd_saida_1_devolv_saida', 'ICMS_SAIDA_UNI',
       'ULT_ICMS_SAIDA_UNI', 'ICMS_SAIDA', 'ICMS_TOT_PCAT', 'VL_CONFR_0',
       'VLR_CONFR_UNIT', 'ULT_VLR_CONFR_UNIT', 'VLR_CONFR_1', 'QTD_SALDO',
       'ICMS_TOT_SALDO', 'VLR_RESSARCIMENTO', 'VLR_COMPLEMENTO', 'COD_LEGAL',
       'COD_LEGAL_PCAT', 'VLR_CONFR_PCAT', 'SALDO_FINAL_MES_QTD',
       'SALDO_FINAL_MES_ICMS', 'ALIQUOTA', 'VALOR',
       'Valor Base Cálculo ICMS ST Retido Operação Anterior',
       'Valor Complementar', 'Valor ICMS Substituição Tributária',
       'Valor ICMS Operação', 'ICMS_TOT', 'DESCRICAO', 'CODIGO_BARRA',
       'UNIDADE', 'N C M', 'CEST', 'CNPJ EMITENTE', 'CNPJ DESTINATARIO',
       'Data'],
      dtype='object')

In [75]:
planilha[planilha['CODIGO_BARRA'] == 'SEM GTIN      ']

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,35191271605265000242590003519330916642468850,2019-12-23,5405,1,18,1,1,"1,000",5.0,14.234097,...,0.0,0.0000,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00,122019
1,35200171605265000242590006721510483174585808,2020-01-27,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Benegrip Multi Dia 4S,SEM GTIN,UN,30045090,1300101,71605265000242,0.000000e+00,012020
2,35200171605265000242590006721510484210083682,2020-01-27,5405,15,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Sal De Fruta Tradicional 100g,SEM GTIN,UN,30049029,1300301,71605265000242,0.000000e+00,012020
3,35200171605265000242590006721510484210083682,2020-01-27,5405,16,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Esogastro Ibp 20mg 84S,SEM GTIN,UN,30049099,1300301,71605265000242,0.000000e+00,012020
4,35200171605265000242590006721510484210083682,2020-01-27,5405,17,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Dexilant 60mg 60S Takeda,SEM GTIN,UN,30039033,<NA>,71605265000242,0.000000e+00,012020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,35200527849963000110550010007887601007861750,2020-05-19,1403,1,687,0,1,"3,000",NaN,NaN,...,0.0,9.1872,Vagisoft Gel 30g Hertz,SEM GTIN,UN,30067000,2000500,71605265000242,0.000000e+00,052020
415725,35191171605265000242590006721510366000864218,2019-11-18,5405,5,932202,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Xigduo Xr 10mg/1000mg 5x6S,SEM GTIN,UN,30049059,<NA>,71605265000242,7.160527e+13,112019
416196,35200771605265000242590006721510688082806528,2020-07-21,5405,8,933242,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Keppra 250mg 30S C1,SEM GTIN,UN,30045040,1300101,71605265000242,7.160527e+13,072020
416214,35201071605265000242590003519331264998343573,2020-10-27,5405,7,933242,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Keppra 250mg 30S C1,SEM GTIN,UN,30045040,1300101,71605265000242,7.160527e+13,102020


In [76]:
planilha['CODIGO_BARRA'] = planilha['CODIGO_BARRA'].replace('SEM GTIN      ', np.nan)
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,35191271605265000242590003519330916642468850,2019-12-23,5405,1,18,1,1,"1,000",5.0,14.234097,...,0.0,0.0,Benegrip Multi Dia 4S,NaN,UN,30045090,1300101,71605265000242,0.000000e+00,122019
1,35200171605265000242590006721510483174585808,2020-01-27,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Benegrip Multi Dia 4S,NaN,UN,30045090,1300101,71605265000242,0.000000e+00,012020
2,35200171605265000242590006721510484210083682,2020-01-27,5405,15,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Sal De Fruta Tradicional 100g,NaN,UN,30049029,1300301,71605265000242,0.000000e+00,012020
3,35200171605265000242590006721510484210083682,2020-01-27,5405,16,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Esogastro Ibp 20mg 84S,NaN,UN,30049099,1300301,71605265000242,0.000000e+00,012020
4,35200171605265000242590006721510484210083682,2020-01-27,5405,17,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,Dexilant 60mg 60S Takeda,NaN,UN,30039033,<NA>,71605265000242,0.000000e+00,012020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416286,35201271605265000242590003519331309428707136,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Levoid 88mcg 30S Ache,7896658008801,UN,30043981,1300300,71605265000242,7.160527e+13,122020
416287,35201271605265000242590006721510835754036640,2020-12-15,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Fenobarbital U.Q 100mg 30S C1,7896006237372,UN,30049069,1300200,71605265000242,7.160527e+13,122020
416288,35201271605265000242590006721510835754036640,2020-12-15,5405,3,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Alprazolam Leg 2mg 30S B1,7894916149143,UN,30049064,1300200,71605265000242,7.160527e+13,122020
416289,35201271605265000242590006721510844913169435,2020-12-20,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,Ceftriaxona Eur 1g Im 1ml,7891317007836,UN,30042059,1300200,71605265000242,7.160527e+13,122020


In [77]:
list(planilha['CODIGO_BARRA'].unique())

[nan,
 '7896261000223',
 '7896026300223',
 '7891106906944',
 '7896658018732',
 '7891058431037',
 '7896658002076',
 '7896658008801',
 '7896548111727',
 '7896070603271',
 '7891058019365',
 '7896658001390',
 '7891058059149',
 '5702191007985',
 '7891106906012',
 '7896658015427',
 '7896181907534',
 '7896658004834',
 '7896658009631',
 '7896082900351',
 '7896658003653',
 '7890282801210',
 '7896658005336',
 '4260085521425',
 '7898040321390',
 '7898040321437',
 '7898037080774',
 '7896658028366',
 '7896658023125',
 '7897572001923',
 '7898040320683',
 '7891142130822',
 '7897322709420',
 '7896094903067',
 '7897322701998',
 '7898949988335',
 '7898133130403',
 '7894916509350',
 '8901580008900',
 '7891142202833',
 '7891142165770',
 '7896180300152',
 '7896006290322',
 '7896676402575',
 '7896317904802',
 '7891317469283',
 '7891317422042',
 '7891317461089',
 '7898148298914',
 '7897076907615',
 '7899547501469',
 '7891317001438',
 '7896523208534',
 '7896004712581',
 '7896006212300',
 '7891317007829',
 '78

In [78]:
# Geração do dataset base para geração da 1100

cols_1100 = ['CHV_DOC', 'DATA', 'NUM_ITEM', 'IND_OPER', 'COD_ITEM', 'CFOP','QTD_CAT', 'ICMS_TOT_PCAT', 'VLR_CONFR_PCAT', 'COD_LEGAL_PCAT']

df_1100 = planilha.copy()
df_1100 = df_1100[cols_1100]
df_1100['Data'] = df_1100['DATA']
df_1100['DATA'] = df_1100['DATA'].astype(str)
df_1100['DATA'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%d%m%Y') for x in df_1100['DATA']]
#df_1100.rename(coluns={'Número Item': 'NUM_ITEM'}, inplace=True)


df_1100['QTD_FIN'] = df_1100['QTD_CAT']
df_1100['COD_REG'] = ['1100']*df_1100.shape[0]
df_1100 = df_1100[['COD_REG','CHV_DOC','Data','DATA','NUM_ITEM','IND_OPER','COD_ITEM','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,COD_ITEM,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT
0,1100,35191271605265000242590003519330916642468850,2019-12-23,23122019,1,1,18,5405,"1,000",NaN,NaN,0
1,1100,35200171605265000242590006721510483174585808,2020-01-27,27012020,1,1,18,5405,"1,000",NaN,NaN,0
2,1100,35200171605265000242590006721510484210083682,2020-01-27,27012020,15,1,18,5405,"1,000",NaN,NaN,0
3,1100,35200171605265000242590006721510484210083682,2020-01-27,27012020,16,1,18,5405,"1,000",NaN,NaN,0
4,1100,35200171605265000242590006721510484210083682,2020-01-27,27012020,17,1,18,5405,"1,000",NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
416286,1100,35201271605265000242590003519331309428707136,2020-12-15,15122020,1,1,935908,5405,"1,000",NaN,4.119925,1
416287,1100,35201271605265000242590006721510835754036640,2020-12-15,15122020,1,1,935908,5405,"1,000",NaN,4.120000,1
416288,1100,35201271605265000242590006721510835754036640,2020-12-15,15122020,3,1,935908,5405,"1,000",NaN,4.120000,1
416289,1100,35201271605265000242590006721510844913169435,2020-12-20,20122020,1,1,935908,5405,"1,000",NaN,4.120000,1


In [79]:
df_1100['Data'] = df_1100['Data'].astype(str)
df_1100['Data'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m%Y') for x in df_1100['Data']]
df_1100['Data']

0         122019
1         012020
2         012020
3         012020
4         012020
           ...  
416286    122020
416287    122020
416288    122020
416289    122020
416290    122020
Name: Data, Length: 416291, dtype: object

In [80]:
df_1100.dtypes

COD_REG            object
CHV_DOC            object
Data               object
DATA               object
NUM_ITEM            int64
IND_OPER            int64
COD_ITEM            int64
CFOP               object
QTD_FIN            object
ICMS_TOT_PCAT     float64
VLR_CONFR_PCAT    float64
COD_LEGAL_PCAT     object
dtype: object

In [81]:
df_1100['QTD_FIN'] = df_1100['QTD_FIN'].replace('nan', np.nan)
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,COD_ITEM,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT
0,1100,35191271605265000242590003519330916642468850,122019,23122019,1,1,18,5405,"1,000",NaN,NaN,0
1,1100,35200171605265000242590006721510483174585808,012020,27012020,1,1,18,5405,"1,000",NaN,NaN,0
2,1100,35200171605265000242590006721510484210083682,012020,27012020,15,1,18,5405,"1,000",NaN,NaN,0
3,1100,35200171605265000242590006721510484210083682,012020,27012020,16,1,18,5405,"1,000",NaN,NaN,0
4,1100,35200171605265000242590006721510484210083682,012020,27012020,17,1,18,5405,"1,000",NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
416286,1100,35201271605265000242590003519331309428707136,122020,15122020,1,1,935908,5405,"1,000",NaN,4.119925,1
416287,1100,35201271605265000242590006721510835754036640,122020,15122020,1,1,935908,5405,"1,000",NaN,4.120000,1
416288,1100,35201271605265000242590006721510835754036640,122020,15122020,3,1,935908,5405,"1,000",NaN,4.120000,1
416289,1100,35201271605265000242590006721510844913169435,122020,20122020,1,1,935908,5405,"1,000",NaN,4.120000,1


In [82]:
# Leitura do arquivo da 1050

arquivo_1050 = pd.read_excel(f'Loja_{loja}/1050_loja_{loja}_v2.xlsx', dtype=str)

In [83]:
mes_ano = []
for date in arquivo_1050['REF']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])
    
arquivo_1050['DATA'] = [datetime.strptime(x, '%Y%m').strftime('%m%Y') for x in arquivo_1050['REF']]
arquivo_1050

,Unnamed: 0,REF,COD_ITEM,QTD_INI,ICMS_INI,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,DATA
0,0,201910,18,5,128,5,128,102019
1,1,201911,18,5,14.2340965327479,5,14.2340965327479,112019
2,2,201912,18,5,14.2340965327479,4,11.4,122019
3,3,202001,18,4,11.4,2,6.37,012020
4,4,202002,18,2,6.37,2,6.37,022020
...,...,...,...,...,...,...,...,...
88150,88150,202008,935908,2,6.02,2,6.02,082020
88151,88151,202009,935908,2,6.02,2,6.02,092020
88152,88152,202010,935908,0,0,0,0,102020
88153,88153,202011,935908,0,0,69,284.25,112020


In [84]:
for date in planilha['Data'].unique():
    print(date)

122019
012020
042020
052020
122020
102019
022020
062020
092020
102020
112020
112019
032020
072020
082020


In [85]:
# Definição de uma função para exportar os
# dataframes de cada documentação em um
# arquivo .txt com a formatação desejada

def dataframe_to_txt_with_crlf(data, path):
    with open(path, 'a', newline='', encoding='utf-8') as f:
        for index, row in data.iterrows():
            row_str = '|'.join(map(lambda x: '' if pd.isna(x) else str(x), row)) + '\r\n'
            f.write(row_str)

In [86]:
def format_number(value):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')  # Set your desired locale
    formatted_value = locale.format('%.2f', value, grouping=True)
    formatted_value = str(formatted_value).replace('.', '')
    return formatted_value

In [92]:
df_0150 = pd.read_excel(f'Loja_{loja}/0150_{loja}_v2.xlsx', dtype=str)
df_0150 = df_0150[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'COD_MUN', 'IE', 'DATA']]
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,0150,CLI751982895,Simone regina lopes,1058,NaN,17309032829,3550308,NaN,2019-10-01 00:00:00
1,0150,FOR000008654,Ana cristina azevedo teixeira,1058,NaN,09940419856,3550308,NaN,2019-10-01 00:00:00
2,0150,FOR000014670,Jose edson rodrigues,1058,NaN,99255979868,3550308,NaN,2019-10-01 00:00:00
3,0150,FOR751880776,Mei lim de jesus shen,1058,NaN,11438697880,3550308,NaN,2019-10-01 00:00:00
4,0150,FOR751944941,Gilberto lopes,1058,NaN,21302995804,3550308,NaN,2019-10-01 00:00:00
...,...,...,...,...,...,...,...,...,...
694,0150,CLI000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,2020-12-01 00:00:00
695,0150,FOR000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,2020-12-01 00:00:00
696,0150,CLI000000498,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004825,NaN,3550308,128110725115,2020-12-01 00:00:00
697,0150,FOR000001611,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265006283,NaN,3508504,234137029119,2020-12-01 00:00:00


In [93]:
#df_0150 = df_0150[df_0150['Nome da Origem'] == f'Loja {loja}']
df_0150['DATA'] = df_0150['DATA'].str.slice(0, 10).astype(str)
df_0150['DATA'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m%Y') for x in df_0150['DATA']]
df_0150['CNPJ'] = df_0150['CNPJ'].replace('', np.nan)
df_0150['IE'] = df_0150['IE'].replace('', np.nan)
df_0150['COD_PART'] = df_0150['COD_PART'].str[3:]
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,0150,751982895,Simone regina lopes,1058,NaN,17309032829,3550308,NaN,102019
1,0150,000008654,Ana cristina azevedo teixeira,1058,NaN,09940419856,3550308,NaN,102019
2,0150,000014670,Jose edson rodrigues,1058,NaN,99255979868,3550308,NaN,102019
3,0150,751880776,Mei lim de jesus shen,1058,NaN,11438697880,3550308,NaN,102019
4,0150,751944941,Gilberto lopes,1058,NaN,21302995804,3550308,NaN,102019
...,...,...,...,...,...,...,...,...,...
694,0150,000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,122020
695,0150,000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,122020
696,0150,000000498,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004825,NaN,3550308,128110725115,122020
697,0150,000001611,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265006283,NaN,3508504,234137029119,122020


In [94]:
#df_0150 = df_0150[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'IE', 'COD_MUN']]
df_0150 = df_0150[df_0150['CNPJ'].notnull()]
df_0150 = df_0150.drop_duplicates()
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
34,0150,000002026,Panpharma Distribuidora de Medicamentos Ltda,1058,01206820000520,NaN,3525904,407534370116,102019
35,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,NaN,3543402,639117330119,102019
36,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,5002704,283341718,102019
37,0150,751983304,DELTA HIGIENE LTDA,1058,08894541000112,NaN,3513009,278205571117,102019
38,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,3549904,645539438119,102019
...,...,...,...,...,...,...,...,...,...
693,0150,000000452,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000242,NaN,3550308,145717935110,122020
694,0150,000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,122020
696,0150,000000498,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004825,NaN,3550308,128110725115,122020
697,0150,000001611,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265006283,NaN,3508504,234137029119,122020


In [96]:
mask_duplicates = df_0150[df_0150['DATA'] == '112019'].duplicated(subset='CNPJ', keep='first')
# Invert the mask to keep the first occurrence of each unique 'CNPJ' value
mask_to_keep = ~mask_duplicates

# Apply the mask to the DataFrame to keep the desired rows for the current 'PERIODO'
filtered_df = df_0150[(df_0150['DATA'] == '112019') | mask_to_keep]
filtered_df

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
74,0150,000002026,Panpharma Distribuidora de Medicamentos Ltda,1058,01206820000520,NaN,3525904,407534370116,112019
75,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,NaN,3543402,639117330119,112019
76,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,5002704,283341718,112019
77,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,3549904,645539438119,112019
78,0150,000002004,DHV Distribuidora Hospitalar do Vale Ltda - EPP,1058,23276873000127,NaN,3549904,645739349118,112019
79,0150,751872165,Conde atacadista LTDA,1058,27849963000110,NaN,3508504,234086958110,112019
80,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,3508504,234086958110,112019
81,0000,,1,01122019,31122019,Master Formula Farmacia de Manipulacao LTDA,NaN,71605265000242,112019
82,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,3506003,209017636112,112019
83,0150,000002029,PROFARMA Distribuidora de Produtos Farmaceutic...,1058,45453214001042,NaN,3548906,637117955110,112019


In [97]:
filtered_df['CNPJ'].unique()

array(['01206820000520', '05651966000455', '07259917000154',
       '10328983000143', '23276873000127', '27849963000110', '31122019',
       '44463156000184', '45453214001042', '61068276001682',
       '61940292000137', '71605265000242', '71605265000323',
       '71605265003349', '71605265003853', '71605265003934',
       '71605265004582', '71605265004663', '97554090000102'], dtype=object)

In [99]:
df_0150_final = pd.DataFrame()

# Get the unique 'PERIODO' values
unique_periodos = df_0150['DATA'].unique()

# Iterate through unique 'PERIODO' values
for periodo in unique_periodos:
    # Filter the DataFrame for the current 'PERIODO' value
    filtered_df = df_0150[df_0150['DATA'] == periodo]
    
    # Drop duplicates in the 'CNPJ' column for the current 'PERIODO'
    filtered_df = filtered_df.drop_duplicates(subset='CNPJ', keep='first')
    
    # Append the filtered results to the final DataFrame
    df_0150_final = df_0150_final.append(filtered_df, ignore_index=True)

# Reset the index of the final DataFrame
df_0150_final.reset_index(drop=True, inplace=True)
df_0150_final = df_0150_final[df_0150_final['REG'] == '0150']
df_0150_final

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\575281542.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_0150_final = df_0150_final.append(filtered_df, ignore_index=True)


,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,0150,000002026,Panpharma Distribuidora de Medicamentos Ltda,1058,01206820000520,NaN,3525904,407534370116,102019
1,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,NaN,3543402,639117330119,102019
2,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,5002704,283341718,102019
3,0150,751983304,DELTA HIGIENE LTDA,1058,08894541000112,NaN,3513009,278205571117,102019
4,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,3549904,645539438119,102019
...,...,...,...,...,...,...,...,...,...
244,0150,000000452,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000242,NaN,3550308,145717935110,122020
245,0150,000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,122020
246,0150,000000498,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004825,NaN,3550308,128110725115,122020
247,0150,000001611,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265006283,NaN,3508504,234137029119,122020


In [100]:
df_0150_final = df_0150_final.drop_duplicates()
df_0150_final[df_0150_final['IE'].notnull()]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,0150,000002026,Panpharma Distribuidora de Medicamentos Ltda,1058,01206820000520,NaN,3525904,407534370116,102019
1,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,NaN,3543402,639117330119,102019
2,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,5002704,283341718,102019
3,0150,751983304,DELTA HIGIENE LTDA,1058,08894541000112,NaN,3513009,278205571117,102019
4,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,3549904,645539438119,102019
...,...,...,...,...,...,...,...,...,...
244,0150,000000452,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000242,NaN,3550308,145717935110,122020
245,0150,000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,122020
246,0150,000000498,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004825,NaN,3550308,128110725115,122020
247,0150,000001611,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265006283,NaN,3508504,234137029119,122020


In [102]:
df_cnpj_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = ['102019', '112019', '122019',
        '012020', '022020', '032020', '042020',
        '052020', '062020', '072020', '082020',
        '092020', '102020', '112020', '122020']
for date in dates:
    for cnpj in df_0150_final[df_0150_final['DATA'] == date]['CNPJ'].unique():
        if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]) > 1:
            df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]
            df_cnpj_repetido = pd.concat([df_cnpj_repetido, df])
            
df_cnpj_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA


In [103]:
df_ie_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = ['102019', '112019', '122019',
        '012020', '022020', '032020', '042020',
        '052020', '062020', '072020', '082020',
        '092020', '102020', '112020', '122020']
for date in dates:
    for ie in df_0150_final[df_0150_final['DATA'] == date]['IE'].unique():
        if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]) > 1:
            df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]
            df_ie_repetido = pd.concat([df_ie_repetido, df])
            
df_ie_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA


In [104]:
# Iterate through each file in the folder
df_0000 = pd.read_excel(f'Loja_{loja}/0000_{loja}.xlsx', dtype=str)

df_0000['PERIODO'] = df_0000['PERIODO'].astype(str).str.slice(2, 8)
df_0000['PERIODO'] = [datetime.strptime(x, '%m%Y').strftime('%m%Y') for x in df_0000['PERIODO']]
df_0000['COD_VER'] = ['01'] * df_0000.shape[0]
df_0000['COD_FIN'] = ['00'] * df_0000.shape[0]
df_0000 = df_0000[['REG', 'PERIODO', 'NOME', 'CNPJ', 'IE',
        'COD_MUN', 'COD_VER', 'COD_FIN']]

df_0000

,REG,PERIODO,NOME,CNPJ,IE,COD_MUN,COD_VER,COD_FIN
0,0000,102019,Master Formula Farmacia de Manipulacao LTDA,71605265000242,145717935110,3550308,01,00
1,0000,112019,Master Formula Farmacia de Manipulacao LTDA,71605265000242,145717935110,3550308,01,00
2,0000,122019,Master Formula Farmacia de Manipulacao LTDA,71605265000242,145717935110,3550308,01,00
3,0000,012020,Master Formula Farmacia de Manipulacao LTDA,71605265000242,145717935110,3550308,01,00
4,0000,022020,Master Formula Farmacia de Manipulacao LTDA,71605265000242,145717935110,3550308,01,00
5,0000,032020,Master Formula Farmacia de Manipulacao LTDA,71605265000242,145717935110,3550308,01,00
6,0000,042020,Master Formula Farmacia de Manipulacao LTDA,71605265000242,145717935110,3550308,01,00
7,0000,052020,Master Formula Farmacia de Manipulacao LTDA,71605265000242,145717935110,3550308,01,00
8,0000,062020,Master Formula Farmacia de Manipulacao LTDA,71605265000242,145717935110,3550308,01,00
9,0000,072020,Master Formula Farmacia de Manipulacao LTDA,71605265000242,145717935110,3550308,01,00


In [105]:
for date in planilha['Data'].unique():
    print(date)

122019
012020
042020
052020
122020
102019
022020
062020
092020
102020
112020
112019
032020
072020
082020


In [106]:
df_0150_final['COD_MUN'] = pd.to_numeric(df_0150_final['COD_MUN'])
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,0150,000002026,Panpharma Distribuidora de Medicamentos Ltda,1058,01206820000520,NaN,3525904,407534370116,102019
1,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,NaN,3543402,639117330119,102019
2,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,5002704,283341718,102019
3,0150,751983304,DELTA HIGIENE LTDA,1058,08894541000112,NaN,3513009,278205571117,102019
4,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,3549904,645539438119,102019
...,...,...,...,...,...,...,...,...,...
244,0150,000000452,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000242,NaN,3550308,145717935110,122020
245,0150,000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,122020
246,0150,000000498,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004825,NaN,3550308,128110725115,122020
247,0150,000001611,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265006283,NaN,3508504,234137029119,122020


In [107]:
def fill_missing_ie(group):
    non_null_ie = group.dropna()['IE']
    if not non_null_ie.empty:
        group['IE'] = non_null_ie.iloc[0]
    elif group['IE'].isnull().all():
        group['IE'] = np.nan
    return group

# Group by 'CNPJ' and apply the custom function
df_0150_final_new = df_0150_final.groupby('CNPJ', group_keys=False).apply(fill_missing_ie)

df_0150_final_new

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,0150,000002026,Panpharma Distribuidora de Medicamentos Ltda,1058,01206820000520,NaN,3525904,407534370116,102019
1,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,NaN,3543402,639117330119,102019
2,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,5002704,283341718,102019
3,0150,751983304,DELTA HIGIENE LTDA,1058,08894541000112,NaN,3513009,278205571117,102019
4,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,3549904,645539438119,102019
...,...,...,...,...,...,...,...,...,...
244,0150,000000452,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000242,NaN,3550308,145717935110,122020
245,0150,000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,122020
246,0150,000000498,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004825,NaN,3550308,128110725115,122020
247,0150,000001611,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265006283,NaN,3508504,234137029119,122020


In [108]:
df_0150_final = df_0150_final[df_0150_final['IE'].notnull()]
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,0150,000002026,Panpharma Distribuidora de Medicamentos Ltda,1058,01206820000520,NaN,3525904,407534370116,102019
1,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,NaN,3543402,639117330119,102019
2,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,5002704,283341718,102019
3,0150,751983304,DELTA HIGIENE LTDA,1058,08894541000112,NaN,3513009,278205571117,102019
4,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,3549904,645539438119,102019
...,...,...,...,...,...,...,...,...,...
244,0150,000000452,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000242,NaN,3550308,145717935110,122020
245,0150,000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,122020
246,0150,000000498,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004825,NaN,3550308,128110725115,122020
247,0150,000001611,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265006283,NaN,3508504,234137029119,122020


In [109]:
df_0150_final.loc[(df_0150_final['CNPJ'].notnull()) & (df_0150_final['IE'].isnull())]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA


In [110]:
df_0150_final['CNPJ'] = df_0150_final['CNPJ'].apply(lambda x: str(x).zfill(14) if len(str(x)) < 14 else str(x))
df_0150_final

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\2152152490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150_final['CNPJ'] = df_0150_final['CNPJ'].apply(lambda x: str(x).zfill(14) if len(str(x)) < 14 else str(x))


,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,0150,000002026,Panpharma Distribuidora de Medicamentos Ltda,1058,01206820000520,NaN,3525904,407534370116,102019
1,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,NaN,3543402,639117330119,102019
2,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,5002704,283341718,102019
3,0150,751983304,DELTA HIGIENE LTDA,1058,08894541000112,NaN,3513009,278205571117,102019
4,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,3549904,645539438119,102019
...,...,...,...,...,...,...,...,...,...
244,0150,000000452,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000242,NaN,3550308,145717935110,122020
245,0150,000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,122020
246,0150,000000498,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004825,NaN,3550308,128110725115,122020
247,0150,000001611,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265006283,NaN,3508504,234137029119,122020


In [111]:
# Função para verificar o dígito verificador de um CNPJ
def verifica_digito_verificador(cnpj):
    cnpj = [int(d) for d in str(cnpj) if d.isdigit()]  # Converte o CNPJ em uma lista de números inteiros
    if len(cnpj) != 14:
        return False  # CNPJ deve ter 14 dígitos

    # Calcula o primeiro dígito verificador
    soma = 0
    peso = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    for i in range(12):
        soma += cnpj[i] * peso[i]
    resto = soma % 11
    if resto < 2:
        digito_verificador1 = 0
    else:
        digito_verificador1 = 11 - resto

    # Calcula o segundo dígito verificador
    soma = 0
    peso.insert(0, 6)
    for i in range(13):
        soma += cnpj[i] * peso[i]
    resto = soma % 11
    if resto < 2:
        digito_verificador2 = 0
    else:
        digito_verificador2 = 11 - resto

    # Verifica se os dígitos verificadores são iguais aos dígitos originais
    return cnpj[-2] == digito_verificador1 and cnpj[-1] == digito_verificador2

# Exemplo de DataFrame
df = df_0150_final.copy()

# Aplica a função verifica_digito_verificador à coluna 'CNPJ' do DataFrame
df['Dígito_Verificador_Válido'] = df['CNPJ'].apply(verifica_digito_verificador)

# Exibe o DataFrame com a nova coluna
df

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA,Dígito_Verificador_Válido
0,0150,000002026,Panpharma Distribuidora de Medicamentos Ltda,1058,01206820000520,NaN,3525904,407534370116,102019,True
1,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,NaN,3543402,639117330119,102019,True
2,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,5002704,283341718,102019,True
3,0150,751983304,DELTA HIGIENE LTDA,1058,08894541000112,NaN,3513009,278205571117,102019,True
4,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,3549904,645539438119,102019,True
...,...,...,...,...,...,...,...,...,...,...
244,0150,000000452,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000242,NaN,3550308,145717935110,122020,True
245,0150,000000453,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000323,NaN,3550308,145800771110,122020,True
246,0150,000000498,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004825,NaN,3550308,128110725115,122020,True
247,0150,000001611,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265006283,NaN,3508504,234137029119,122020,True


In [112]:
df['Dígito_Verificador_Válido'].unique()

array([ True])

In [113]:
unique_cod_items = planilha[planilha['Data'] == '102019']['COD_ITEM'].unique()
arquivo_1050['COD_ITEM'] = arquivo_1050['COD_ITEM'].astype(int)
filtered = arquivo_1050[arquivo_1050['DATA'] == '102019'][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
filtered

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\641789374.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == '102019'][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]


,Unnamed: 0,REF,COD_ITEM,QTD_INI,ICMS_INI,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,DATA
15,15,201910,26,2,18.53027554695538,6,57.35,102019
45,45,201910,59,1,4.555330674995751,2,11.06,102019
60,60,201910,75,22,15.95439977042259,16,11.64,102019
75,75,201910,125,1,6.577254542934992,1,7.12,102019
90,90,201910,141,1,6.96006262636508,0,0,102019
...,...,...,...,...,...,...,...,...
87960,87960,201910,932905,4,12.53995469617388,4,12.7,102019
88005,88005,201910,933010,1,16.15928430078282,1,21.78,102019
88020,88020,201910,933184,4,6.398380210313083,3,7.779999999999999,102019
88065,88065,201910,933622,1,28.24451344434289,1,25.83,102019


In [114]:
len(filtered['COD_ITEM'].unique())

3288

In [115]:
# Geração dos TXTs da CAT

for date in planilha['Data'].unique():
#for date in ['112019']:
    # Geração da 0000
    df_0000_ = df_0000[df_0000['PERIODO'] == date]
    
    # Geração da 0150

    # df_0150 has originally 8 columns
    df_0150_ = df_0150_final[df_0150_final['DATA'] == date]
    df_0150_ = df_0150_[df_0150_['CNPJ'].notnull()]
    df_0150_ = df_0150_[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF',
                'IE', 'COD_MUN']]
    
    # Geração da 0200
    df_0200 = planilha[planilha['Data'] == date][['COD_ITEM','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']].drop_duplicates()
    df_0200['COD_REG'] = ['0200']*(df_0200.shape[0])
    df_0200 = df_0200[['COD_REG', 'COD_ITEM','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']]
    df_0200['ALIQUOTA'] = df_0200['ALIQUOTA'].astype(float).astype(int)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].astype(str)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].replace('nan', np.nan)
    df_0200.drop_duplicates(subset='COD_ITEM', keep='first', inplace=True)
    
    # Geração da 1050
    unique_cod_items = planilha[planilha['Data'] == date]['COD_ITEM'].unique()
    arquivo_1050['COD_ITEM'] = arquivo_1050['COD_ITEM'].astype(int)
    filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
    filtered

    df_1050 = filtered[['COD_ITEM','QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
    
    df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
    df_1050['QTD_INI'] = df_1050['QTD_INI'].astype(float).astype(int).astype(str) + ',000'
    df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
    df_1050['SALDO_FINAL_MES_QTD'] = df_1050['SALDO_FINAL_MES_QTD'].astype(float).astype(int).astype(str) + ',000'
    df_1050['SALDO_FINAL_MES_ICMS'] = df_1050['SALDO_FINAL_MES_ICMS'].astype(float).map(format_number)
    
    df_1050 = df_1050[['COD_REG', 'COD_ITEM', 'QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
    df_1050.rename(columns={'COD_REG': 'REG'}, inplace=True)
    
    # Geração da 1100
    df_1100_ = df_1100[df_1100['Data'] == date]
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_ = df_1100_[['COD_REG','CHV_DOC','DATA','NUM_ITEM','IND_OPER','COD_ITEM','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]
    
    # Geração do .txt da 0000
    dataframe_to_txt_with_crlf(df_0000_, f'Loja_{loja}/CAT_Loja_{loja}/teste/0000_{date}_v2.txt')
    
    # Geração do .txt da 0150
    dataframe_to_txt_with_crlf(df_0150_, f'Loja_{loja}/CAT_Loja_{loja}/teste/0150_{date}_v2.txt')
    
    # Geração do .txt da 0200
    dataframe_to_txt_with_crlf(df_0200, f'Loja_{loja}/CAT_Loja_{loja}/teste/0200_{date}_v2.txt')
    
    # Geração do .txt da 1050
    dataframe_to_txt_with_crlf(df_1050, f'Loja_{loja}/CAT_Loja_{loja}/teste/1050_{date}_v2.txt')
    
    # Geração do .txt da 1100
    dataframe_to_txt_with_crlf(df_1100_, f'Loja_{loja}/CAT_Loja_{loja}/teste/1100_{date}_v2.txt')
    
    # Geração de um arquivo .txt único,
    # contendo todas as informações das documentações
    # da 0000, 0150, 0200, 1050 e 1100
    # dentro do formato correto

    read_files = glob.glob(f"Loja_{loja}/CAT_Loja_{loja}/teste/*{date}_v2.txt")
    
    with open(f"Loja_{loja}/CAT_Loja_{loja}/cat_loja_{loja}_{date}_v2.txt", "wb") as outfile:
        for f in read_files:
            with open(f, "rb") as infile:
                outfile.write(infile.read())

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7976\1918594264.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [ ]:
end = time()

print(end-start)